In [ ]:
import boto3
import json
# instantiate a new comprehend client by reading credentials from config file
comprehend = boto3.client(service_name='comprehend') 

In [2]:
batch_string=[
    'PAYMENT TO ANZ BANK MERCHANT FEE',
    'PAYMENT TO MYOB AUSTRALIA   MYOB 1-10327685617'
    'ANZ INTERNET BANKING FUNDS TFER TRANSFER 866802  TO  015208384929555',
    'ANZ INTERNET BANKING BPAY NEWS - SUBSCRIPTIO            {184786}',
    'VISA DEBIT PURCHASE CARD 8061 GOOGLE*ADWS8409660624 SINGAPORE',
    'VISA DEBIT PURCHASE CARD 8061 RENTALCARTOLLPH131865 PARRAMATTA',
    'Gutschrift Onlinebanking ner 2014 BG/000000217 OPSKATWW AT916000050110110437 Piratenpartei Österreichs Mitgliedsbeiträge LO-Anteil Jänner 2014',
    'SWITCH Energievertriebsgesellschaft Bel. 613661801 EUR 3700 OG/000001242'
 ,'SIX Payment Services (Europe) S.A.'
,'Stadt Wien Buchhaltungsabteilung 33 POS 6507 280 K001 26.08. 16:45 JET-TANKSTELLEMUENCHEN80687'
,'Entgelt für Buchungsposten und Belege'
,'Entgelt für Manipulation'
,'Entgelt für Buchungsposten und Belege'
,'INTER-BANK CREDIT, SO1000 Grumpy Donuts SOL Products Pty'
             ]
len(batch_string)

13

In [16]:
comprehend.detect_entities(Text='SWITCH Energievertriebsgesellschaft Bel. 613661801 EUR 3700 OG/000001242',LanguageCode='en')

{'Entities': [{'Score': 0.8370881676673889,
   'Type': 'ORGANIZATION',
   'Text': 'SWITCH Energievertriebsgesellschaft',
   'BeginOffset': 0,
   'EndOffset': 35},
  {'Score': 0.9381502270698547,
   'Type': 'OTHER',
   'Text': '613661801',
   'BeginOffset': 41,
   'EndOffset': 50},
  {'Score': 0.7416148781776428,
   'Type': 'OTHER',
   'Text': 'EUR 3700',
   'BeginOffset': 51,
   'EndOffset': 59},
  {'Score': 0.5299660563468933,
   'Type': 'OTHER',
   'Text': 'OG/000001242',
   'BeginOffset': 60,
   'EndOffset': 72}],
 'ResponseMetadata': {'RequestId': '322584e8-8610-49ef-9998-3554affdb2d2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '322584e8-8610-49ef-9998-3554affdb2d2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '428',
   'date': 'Thu, 08 Oct 2020 11:48:40 GMT'},
  'RetryAttempts': 0}}

In [3]:
s=batch_string[8]
s=' '.join(x for x in s.split() if x)
print(s)
detlang=comprehend.detect_dominant_language(Text=s)['Languages'][0]['LanguageCode']
print(detlang)
detent=comprehend.detect_entities(Text = s, LanguageCode=detlang)['Entities']
detent

Stadt Wien Buchhaltungsabteilung 33 POS 6507 280 K001 26.08. 16:45 JET-TANKSTELLEMUENCHEN80687
de


[{'Score': 0.6825277209281921,
  'Type': 'ORGANIZATION',
  'Text': 'Stadt Wien Buchhaltungsabteilung',
  'BeginOffset': 0,
  'EndOffset': 32},
 {'Score': 0.8785567879676819,
  'Type': 'OTHER',
  'Text': '33 POS 6507 280 K001',
  'BeginOffset': 33,
  'EndOffset': 53},
 {'Score': 0.9966641664505005,
  'Type': 'DATE',
  'Text': '26.08. 16:45',
  'BeginOffset': 54,
  'EndOffset': 66}]

In [9]:
detkeyph=comprehend.detect_key_phrases(Text = s, LanguageCode=detlang)['KeyPhrases']
detkeyph

[{'Score': 0.9985055327415466,
  'Text': 'Stadt Wien',
  'BeginOffset': 0,
  'EndOffset': 10},
 {'Score': 0.9356321096420288,
  'Text': 'Buchhaltungsabteilung',
  'BeginOffset': 11,
  'EndOffset': 32},
 {'Score': 0.7592402696609497,
  'Text': '33 POS',
  'BeginOffset': 33,
  'EndOffset': 39},
 {'Score': 0.999977707862854,
  'Text': '6507',
  'BeginOffset': 40,
  'EndOffset': 44},
 {'Score': 0.9838687777519226,
  'Text': '280 K001',
  'BeginOffset': 45,
  'EndOffset': 53},
 {'Score': 0.989812433719635,
  'Text': '26.08. 16:45',
  'BeginOffset': 54,
  'EndOffset': 66}]

In [ ]:
bd=comprehend.batch_detect_entities(TextList = batch_string, LanguageCode='de')['ResultList']
bd

In [ ]:
dtbd=pd.DataFrame.from_dict(bd)
# dtbd=dtbd.Entities.apply(lambda x : pd.json_normalize(x))
dtbd

In [14]:
for kp in detkeyph:
    print(kp['Text'])
    print(comprehend.detect_entities(Text = kp['Text'], LanguageCode=detlang)['Entities'])


Stadt Wien
[{'Score': 0.8093973398208618, 'Type': 'LOCATION', 'Text': 'Stadt Wien', 'BeginOffset': 0, 'EndOffset': 10}]
Buchhaltungsabteilung
[]
33 POS
[{'Score': 0.4266717731952667, 'Type': 'QUANTITY', 'Text': '33', 'BeginOffset': 0, 'EndOffset': 2}]
6507
[{'Score': 0.5639507174491882, 'Type': 'OTHER', 'Text': '6507', 'BeginOffset': 0, 'EndOffset': 4}]
280 K001
[{'Score': 0.5401226282119751, 'Type': 'COMMERCIAL_ITEM', 'Text': '280 K001', 'BeginOffset': 0, 'EndOffset': 8}]
26.08. 16:45
[{'Score': 0.993584394454956, 'Type': 'DATE', 'Text': '26.08. 16:45', 'BeginOffset': 0, 'EndOffset': 12}]


In [15]:
detsyn=comprehend.detect_syntax(Text=s, LanguageCode=detlang)['SyntaxTokens']
detsyn

[{'TokenId': 1,
  'Text': 'Stadt',
  'BeginOffset': 0,
  'EndOffset': 5,
  'PartOfSpeech': {'Tag': 'NOUN', 'Score': 0.9006140232086182}},
 {'TokenId': 2,
  'Text': 'Wien',
  'BeginOffset': 6,
  'EndOffset': 10,
  'PartOfSpeech': {'Tag': 'PROPN', 'Score': 0.998957633972168}},
 {'TokenId': 3,
  'Text': 'Buchhaltungsabteilung',
  'BeginOffset': 11,
  'EndOffset': 32,
  'PartOfSpeech': {'Tag': 'NOUN', 'Score': 0.5653969049453735}},
 {'TokenId': 4,
  'Text': '33',
  'BeginOffset': 33,
  'EndOffset': 35,
  'PartOfSpeech': {'Tag': 'NUM', 'Score': 0.9811906218528748}},
 {'TokenId': 5,
  'Text': 'POS',
  'BeginOffset': 36,
  'EndOffset': 39,
  'PartOfSpeech': {'Tag': 'PROPN', 'Score': 0.9334350228309631}},
 {'TokenId': 6,
  'Text': '6507',
  'BeginOffset': 40,
  'EndOffset': 44,
  'PartOfSpeech': {'Tag': 'PROPN', 'Score': 0.9570008516311646}},
 {'TokenId': 7,
  'Text': '280',
  'BeginOffset': 45,
  'EndOffset': 48,
  'PartOfSpeech': {'Tag': 'PROPN', 'Score': 0.9490844011306763}},
 {'TokenId': 8

In [ ]:
detsyn[0]

In [ ]:
import pandas as pd
pd.DataFrame.from_dict(detsyn[0])

In [ ]:
print('Calling DetectDominantLanguage')

print('english_string result:')
# json.dumps() writes JSON data to a Python string
print(json.dumps(comprehend.detect_dominant_language(Text = english_string), sort_keys=True, indent=4))

print('\n spanish_string result:')
print(json.dumps(comprehend.detect_dominant_language(Text = spanish_string), sort_keys=True, indent=4))
print('End of DetectDominantLanguage\n')

print('Calling DetectEntities')
# json.dumps() writes JSON data to a Python string
print(json.dumps(comprehend.detect_entities(Text = english_string, LanguageCode='en'), sort_keys=True, indent=4))
print(json.dumps(comprehend.detect_entities(Text = spanish_string, LanguageCode='de'), sort_keys=True, indent=4))
print('End of DetectEntities\n')

In [ ]:
print('Calling BatchDetectDominantLanguage')

print('english_string_list results:')
# json.dumps() writes JSON data to a Python string
print(json.dumps(comprehend.batch_detect_dominant_language(TextList = batch_string), sort_keys=True, indent=4))

# print('\nspanish_string_list results:')
# print(json.dumps(comprehend.batch_detect_dominant_language(TextList = batch_string), sort_keys=True, indent=4))
print('End of BatchDetectDominantLanguage\n')

In [ ]:
# print(json.dumps(comprehend.batch_detect_entities(TextList = batch_string, LanguageCode='de'), sort_keys=True, indent=4))
comprehend.batch_detect_entities(TextList = batch_string, LanguageCode='de')


In [ ]:
# dtkp=comprehend.batch_detect_key_phrases(TextList = batch_string, LanguageCode='de')['ResultList']
# dtkp=pd.DataFrame.from_dict(dtkp)
# pd.read_json(dtkp,orient='records')
dtkp

In [ ]:
for s in batch_string:
    print(json.dumps(comprehend.detect_entities(Text = s, LanguageCode='de'), sort_keys=True, indent=4))

In [ ]:
print(s)
detent=comprehend.detect_entities(Text = s, LanguageCode='de')
print(detent)

# detent.keys()
print(detent.values())
print(detent['Entities'])
len(detent['Entities'])

In [ ]:
s

In [ ]:
print('Calling DetectKeyPhrases')
# json.dumps() writes JSON data to a Python string
for s in batch_string:
    print(json.dumps(comprehend.detect_key_phrases(Text = s, LanguageCode='en'), sort_keys=True, indent=4))
print('End of DetectKeyPhrases\n')

In [ ]:
# provide a text string to analyze
english_string = 'Today is my birthday, I am so happy.'

print('Calling DetectSentiment')
# json.dumps() writes JSON data to a Python string
print('english_string results:')
print(json.dumps(comprehend.detect_sentiment(Text = english_string, LanguageCode='en'), sort_keys=True, indent=4))
print('End of DetectSentiment\n')

In [ ]:
# LOAD AND READ DATA FROM S3 BUCKET
# Setup a region
region = 'us-east-2'
# Create an S3 client
s3 = boto3.client('s3',region_name = region)

# Creates a variable with the bucket name
bucket_name = 'dm-sagemaker-s3' #'<insert a unique bucket name>' 

# Create a location Constraint
location = {'LocationConstraint': region}
# Creates a new bucket 
s3.create_bucket(Bucket=bucket_name,CreateBucketConfiguration=location)
# Create a list of file names
filenames_list = ['doc-topics.csv', 'topic-terms.csv']

# Iterates on each file in the  filenames_list
for filename in filenames_list:
    # Uploads each CSV to the created bucket
    s3.upload_file(filename, bucket_name, filename)
    # checks if the filename is 'doc-topics.csv'
    if filename == 'doc-topics.csv':
        # gets the 'doc-topics.csv' file as an object
        obj = s3.get_object(Bucket=bucket_name, Key=filename)
        # reads the csv and assigns to doc_topics 
        doc_topics = pd.read_csv(obj['Body'])
    else:
        obj = s3.get_object(Bucket=bucket_name, Key=filename)
        topic_terms = pd.read_csv(obj['Body'])

# merge files on topic column to obtain the most common terms per document
merged_df = pd.merge(doc_topics, topic_terms, on='topic')

# print the merged_df to the console
print(merged_df)

In [ ]:
import boto3
import json

def lambda_handler(event, context):
    # create a s3 object
    s3 = boto3.client("s3")
    
    # check if an event is True 
    if event:
        # enter the specific bucket name to the 'bucket' variable
        bucket = "<input bucket name>"
        
        # event – AWS Lambda uses this parameter to pass in event data to the handler. 
        # This parameter is usually of the Python dict type. It can also be list, str, int, float, or NoneType type
        text_file_obj = event["Records"][0]
        # assign the uploaded text file name to the 'filename' variable 
        filename = str(text_file_obj['s3']['object']['key'])
        
        # print the filename 
        print("filename: ", filename)

        # create the file object
        file_obj = s3.get_object(Bucket = bucket, Key = filename)
        
        # access the file_obj's body. Invoke the read() function and convert to a str object. 
        # assign to the variable 'body_str_obj' 
        body_str_obj = str(file_obj['Body'].read())
        
        # create a comprehend object
        comprehend = boto3.client(service_name="comprehend")
        
        # call detect_sentiment()
        sentiment_response = comprehend.detect_sentiment(Text = body_str_obj, LanguageCode = "en")
        print("sentiment_response: \n", sentiment_response)
        
        # call detect_entities()
        entity_response = comprehend.detect_entities(Text = body_str_obj, LanguageCode = "en")
        print("\n\nentity_response: \n", entity_response)
        
        # call detect_key_phrases()
        key_phases_response = comprehend.detect_key_phrases(Text = body_str_obj, LanguageCode = "en") 
        print("\n\nkey_phases_response: \n", key_phases_response)
      
        return {
            'statusCode' :200,
            'body' : json.dumps('Hello from Lambda')
        }